# Analysis

In [1]:
import sys
from pathlib import Path

# Make local packages discoverable
package_path = Path().resolve().parent
sys.path.append(str(package_path))

from statwise.loading import CSVDataLoader
from statwise.cleaning import DataCleaner
from statwise.preparation import DataPreparer
from statwise.selection import UnivariateVariableSelection, NestedCVElasticNetVariableSelection
from statwise.modeling import LogisticRegressionModel, NegativeBinomialRegressionModel

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.base.model import HessianInversionWarning

# Suppress convergence-related warnings since we check convergence explicitly
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=HessianInversionWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning, module='statsmodels')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.model_selection._split')


import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print(f"Random seed set to {SEED}")

Random seed set to 42


## Data Cleaning

We implemented a systematic data cleaning pipeline in Python. Administrative variables (patient identifiers, free-text fields) and clinically irrelevant timestamp variables were removed. Variables were selected based on clinical relevance to the research question.

Height and weight were standardized to metric units. Binary composite outcomes were created when individual outcomes represented different severity levels of the same clinical event. Postoperative sepsis and septic shock were combined into a single "any sepsis" outcome, as both represent points on a severity continuum of the same infectious process. Superficial, deep, and organ/space surgical site infections were combined into a binary SSI indicator. This approach improved statistical power while maintaining clinical interpretability. Component variables were dropped after derivation to avoid redundancy.

Final outcome variables were:
- Surgical length of stay
- Any postoperative sepsis occurrence
- Any surgical site infection
- Readmission related to primary procedure

Predictor variables included patient demographics (age, sex, race/ethnicity), clinical characteristics (ASA classification, functional status, BMI), comorbidities (diabetes, hypertension), preoperative laboratory values (albumin, BUN, creatinine, sodium, bilirubin, hematocrit, WBC, INR, PTT), procedure-related variables (CPT code, incisional negative pressure wound therapy use), and administrative factors (primary payor status, discharge destination). Categorical predictors with sparse categories were consolidated. Variables with natural ordering (ASA classification, functional status) were coded as ordered categorical to preserve clinical relationships.

We excluded variables with more than 20% missing data, as this level of missingness compromises statistical power and imputation reliability. For categorical variables, balance was assessed to ensure adequate representation across categories. We distinguished between truly categorical variables (including nominal text categories and low-cardinality integer codes) and continuous count variables based on their statistical properties. Integer variables were treated as categorical only if they had 10 or fewer unique values or if fewer than 10% of observations had unique values, indicating inherent categorization rather than a continuous scale. This approach correctly identified ordinal ratings and categorical codes while preserving genuine count variables for appropriate statistical handling.

For variables classified as categorical or quasi-categorical, we assessed balance using the second-largest class rather than the smallest category. This criterion retained variables with long tails of rare events that could be collapsed during modeling while excluding those lacking adequate representation in clinically meaningful categories. Variables where the second-largest class had fewer than 10 observations were dropped. Continuous count variables were exempt from balance assessment, as their information content derives from the full distribution rather than discrete category sizes.

In [3]:
file_path = "data/WHIPPLE_DEIDENTIFIED.csv"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    df, log_messages = CSVDataLoader(file_path, log=True).load()

print(df.dtypes)
display(df.head(3))

=== Loading CSV file: data/WHIPPLE_DEIDENTIFIED.csv ===
Initial shape: (165, 126)
Parsed date/time columns: ['Hospital Admission Date', 'Operation Date', 'Procedure/Surgery Start', 'Procedure/Surgery Finish', 'Serum Sodium Date', 'BUN Date', 'Serum Creatinine Date', 'Albumin Date', 'Total Bilirubin Date', 'AST/SGOT Date', 'Alkaline Phosphatase Date', 'WBC Date', 'Hemoglobin Date', 'Hematocrit Date', 'Platelet Count Date', 'INR Date', 'PTT Date', 'Hemoglobin A1c (HbA1c) Date', 'Acute Hospital Discharge Date', 'Date of Death', 'Date of First Readmission', 'Date of First Unplanned Return to OR', 'Date of Second unplanned return']
Date parsing completed | Shape unchanged: (165, 126)
Converted to Int64 (nullable integer): ['INPWT (Yes=1)', 'Concurrent Procedures CPT', 'Serum Sodium', 'AST/SGOT', 'Alkaline Phosphatase', 'Platelet Count', 'Total Blood Transfused (in units)', 'Surgical Length of Stay', 'Hospital Length of Stay', 'Date of Death Unknown', 'Total # of Unplanned Returns to OR', 'F

,INPWT (Yes=1),Age at Time of Surgery,Sex at Birth,Race/Ethnicity,CPT Code,CPT Description,GS (>75) Home Origin Status - Support,Primary Payor Status,Hospital Admission Date,Operation Date,Principal Anesthesia Technique,ASA Classification,Procedure/Surgery Start,Procedure/Surgery Finish,Duration of Surgical Procedure (in minutes),Other Procedures CPT,Concurrent Procedures CPT,Height,Height Unit,Weight,Weight Unit,BMI,Diabetes Mellitus,Tobacco/Nicotine Use,Functional Heath Status,Oxygen Support,Ventilator Dependent,History of Severe COPD,Ascites,Heart Failure,Hypertension requiring medication,Preop Acute Kidney Injury,Most Severe Preop Creatinine Increase,Preop Dialysis,Disseminated Cancer,Immunosuppressive Therapy,Immunosuppressant Category,Bleeding Disorder,Preop RBC Transfusions (72h),Sepsis (SIRS/Sepsis/Septic shock) (48h),Serum Sodium,Serum Sodium Date,BUN,BUN Date,Serum Creatinine,Serum Creatinine Date,Albumin,Albumin Date,Total Bilirubin,Total Bilirubin Date,AST/SGOT,AST/SGOT Date,Alkaline Phosphatase,Alkaline Phosphatase Date,WBC,WBC Date,Hemoglobin,Hemoglobin Date,Hematocrit,Hematocrit Date,Platelet Count,Platelet Count Date,INR,INR Date,PTT,PTT Date,Hemoglobin A1c (HbA1c),Hemoglobin A1c (HbA1c) Date,# of Postop Superficial Incisional SSI,# of Postop Superficial Incisional SSI PATOS,# of Postop Deep Incisional SSI,# of Postop Deep Incisional SSI PATOS,# of Postop Organ/Space SSI,# of Postop Organ/Space SSI PATOS,# of Postop Wound Disruption,# of Postop Pneumonia,# of Postop Pneumonia PATOS,# of Postop Unplanned Intubation,# of Postop Pulmonary Embolism,# of Postop On Ventilator > 48 hours,# of Postop On Ventilator > 48 hours PATOS,# of Postop Renal Insufficiency,# of Postop Dialysis,# of Postop UTI,# of Postop UTI PATOS,# of Stroke/Cerebral Vascular Accident (CVA),# of Cardiac Arrest Requiring CPR,# of Myocardial Infarction,# of Postop Blood Transfusions (72h of surgery start time),Total Blood Transfused (in units),# of Postop Venous Thrombosis Requiring Therapy,# of Postop C. diff,# of Postop Sepsis,# of Postop Sepsis PATOS,# of Postop Septic Shock,# of Postop Septic Shock PATOS,# of Postop Other Occurrences,Acute Hospital Discharge Date,Surgical Length of Stay,Hospital Length of Stay,Hospital Discharge Destination,GS (>=75) Home Discharge - Services,GS (>=75) or EGS Functional Health Status on Discharge,Postoperative ICD10 Code,Postoperative ICD10 Description,Still in Hospital >30 Days,Postop Death w/in 30 days of Procedure,Date of Death,Date of Death Unknown,End of Life/Withdrawal of Care,# of Readmissions w/in 30 days,Date of First Readmission,# of Unplanned Readmissions,# of Readmissions likely related to Primary Procedure,# of Readmissions likely unrelated to Primary Procedure,Total # of Unplanned Returns to OR,Date of First Unplanned Return to OR,First Unplanned Return CPT,First Unplanned Return Related to Primary Procedure,Date of Second unplanned return,Second Unplanned Return CPT,Second Unplanned Return Related to Primary Procedure,30 Day F/U Complete,Pancreatectomy T Stage,Pancreatectomy N Stage,Pancreatectomy M Stage
0,0,78.71,Female,Unknown/Not Reported,48150,"Pancreatectomy, proximal subtotal with total d...",NaN,Unknown,2017-01-18,2017-01-19,General,ASA IV - Severe systemic disease threat to life,2017-01-19 14:29:00,2017-01-19 21:13:00,404,NaN,<NA>,63.0,in,71.66,kg,27.99,Non-insulin,No,Independent,NaN,No,No,No,No,Yes,No,NaN,No,No,No,NaN,No,No,NaN,145,2017-01-19,19.0,2017-01-19,1.57,2017-01-19,3.4,2017-01-19,0.3,2017-01-19,18,2017-01-19,92,2017-01-19,7.5,2017-01-19,NaN,NaT,24.7,2017-01-19,316,2017-01-19,1.11,2017-01-19,26.4,2017-01-19,NaN,NaT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,<NA>,0,0,0,0,0,0,0,2017-02-03,15,16,Home,NaN,NaN,C24.1,Malignant neoplasm of ampulla of Vater,NaN,No,NaT,<NA>,No,1,2017-02-13,1,0,1,<NA>,NaT,<NA>,<NA>,NaT,<NA>,<NA>,Yes,T4,N1,M0/Mx
1,0,60.20,Female,Unknown/Not Reported,48150,"Pancreatectomy, proximal subtotal with total d...",NaN,Unknown,2017-01-24,2017-01-25,General,ASA III 

In [4]:
import json

# Load your metadata
with open("data/column_metadata.json", "r") as f:
    metadata = json.load(f)

# Initialize cleaner with metadata
cleaner = DataCleaner(df, column_metadata=metadata)

# Define derivation functions
def standardize_height_to_cm(df):
    """Convert height to centimeters regardless of unit."""
    result = df["Height"].copy()
    # Only convert inches to cm; cm stays as-is
    inches_mask = df["Height Unit"] == "in"
    result[inches_mask] = result[inches_mask] * 2.54
    # Set invalid units to NaN
    valid_units = df["Height Unit"].isin(["cm", "in"])
    result[~valid_units] = np.nan
    return result.astype("float64")

def standardize_weight_to_kg(df):
    """Convert weight to kilograms regardless of unit."""
    result = df["Weight"].copy()
    # Only convert lbs to kg; kg stays as-is
    lbs_mask = df["Weight Unit"] == "lbs"
    result[lbs_mask] = result[lbs_mask] * 0.453592
    # Set invalid units to NaN
    valid_units = df["Weight Unit"].isin(["kg", "lbs"])
    result[~valid_units] = np.nan
    return result.astype("float64")

def composite_binary_postop_sepsis(df):
    """Binary indicator for any postoperative sepsis occurrence."""
    total_sepsis = df["# of Postop Sepsis"] + df["# of Postop Septic Shock"]
    result = (total_sepsis > 0).map({True: "Yes", False: "No"})
    return result.astype("category")

def composite_binary_postop_ssi(df):
    """Binary indicator for any postoperative surgical site infection."""
    total_ssi = (df["# of Postop Superficial Incisional SSI"] + 
                 df["# of Postop Deep Incisional SSI"] + 
                 df["# of Postop Organ/Space SSI"])
    result = (total_ssi > 0).map({True: "Yes", False: "No"})
    return result.astype("category")

def binary_readmission_pp(df):
    """Binary indicator for readmission related to primary procedure."""
    result = (df["# of Readmissions likely related to Primary Procedure"] > 0).map({True: "Yes", False: "No"})
    return result.astype("category")

def binary_inpwt(df):
    """Binary indicator for incisional negative pressure wound therapy."""
    result = df["INPWT (Yes=1)"].apply(lambda x: "Yes" if x == 1 else ("No" if pd.notna(x) else np.nan))
    return result.astype("category")

# Create derivation map
derivation_map = {
    "Height (cm)": standardize_height_to_cm,
    "Weight (kg)": standardize_weight_to_kg,
    "Postop Sepsis Occurrence": composite_binary_postop_sepsis,
    "Postop SSI Occurrence": composite_binary_postop_ssi,
    "Readmission likely related to Primary Procedure": binary_readmission_pp,
    "INPWT": binary_inpwt
}

# Component columns to drop
drop_components = {
    "Height (cm)": ["Height", "Height Unit"],
    "Weight (kg)": ["Weight", "Weight Unit"],
    "Postop Sepsis Occurrence": ["# of Postop Sepsis", "# of Postop Septic Shock"],
    "Postop SSI Occurrence": ["# of Postop Superficial Incisional SSI", 
                              "# of Postop Deep Incisional SSI", 
                              "# of Postop Organ/Space SSI"],
    "Readmission likely related to Primary Procedure": ["# of Readmissions likely related to Primary Procedure"],
    "INPWT": ["INPWT (Yes=1)"]
}

# Define categorical cleaning
categorical_config = {
    "consolidation_maps": {
        "Race/Ethnicity": {
            "Black or African American": "Black",
            "Asian": "Asian",
            "White": "White",
            "Hispanic or Latino": "Hispanic",
            "Some Other Race": "Other",
            "White,Black or African American,American Indian or Alaska Native": "Other"
        },
        "Primary Payor Status": {
            "Medicare": "Medicare",
            "Medicaid": "Medicaid",
            "Private insurance": "Private Insurance",
            "Other": "Other"
        },
        "Hospital Discharge Destination": {
            "Home/Permanent residence": "Home",
            "Home": "Home",
            "Skilled care, not home": "Skilled Care",
            "Other facility": "Other",
            "Expired": "Expired",
            "Acute care hospital": "Acute Care Hospital",
            "Rehab": "Rehab"
        },
    },
    "ordered_maps": {
        "ASA Classification": [
            "ASA II - Mild systemic disease",
            "ASA III - Severe systemic disease",
            "ASA IV - Severe systemic disease threat to life",
            "ASA V - Moribund"
        ]
    }
}

df, logs = cleaner.run_cleaning_pipeline(
    drop_cols=[
        "Postoperative ICD10 Description",
        "CPT Description"
    ],
    keep_response_cols=[
        "Sepsis (SIRS/Sepsis/Septic shock) (48h)",
        "# of Postop Sepsis",
        "# of Postop Septic Shock",
        "# of Postop Other Occurrences",
        "# of Postop Superficial Incisional SSI",
        "# of Postop Deep Incisional SSI",
        "# of Postop Organ/Space SSI",
        "# of Postop Wound Disruption",
        "# of Readmissions likely related to Primary Procedure",
        "Surgical Length of Stay"
    ],
    derivation_map=derivation_map,
    drop_components=drop_components,
    categorical_config=categorical_config,
    min_minority_count=10,  # Need at least 10 obs in smallest category
    max_missing_rate=0.20   # Drop if >20% missing
)


=== Starting Dataset Cleaning Pipeline ===
Dropping 2 specified columns: ['Postoperative ICD10 Description', 'CPT Description']
Shape changed from (165, 126) to (165, 124)
No datetime columns specified - dropping all 23 datetime columns: ['Hospital Admission Date', 'Operation Date', 'Procedure/Surgery Start', 'Procedure/Surgery Finish', 'Serum Sodium Date', 'BUN Date', 'Serum Creatinine Date', 'Albumin Date', 'Total Bilirubin Date', 'AST/SGOT Date', 'Alkaline Phosphatase Date', 'WBC Date', 'Hemoglobin Date', 'Hematocrit Date', 'Platelet Count Date', 'INR Date', 'PTT Date', 'Hemoglobin A1c (HbA1c) Date', 'Acute Hospital Discharge Date', 'Date of Death', 'Date of First Readmission', 'Date of First Unplanned Return to OR', 'Date of Second unplanned return']
Shape changed from (165, 124) to (165, 101)
Detected 31 response columns from metadata
Keeping response columns: ['Sepsis (SIRS/Sepsis/Septic shock) (48h)', '# of Postop Sepsis', '# of Postop Septic Shock', '# of Postop Other Occurrenc

In [5]:
print(df.dtypes)
display(df.head(3))

Age at Time of Surgery                              float64
Sex at Birth                                       category
Race/Ethnicity                                     category
CPT Code                                              int64
Primary Payor Status                               category
ASA Classification                                 category
Duration of Surgical Procedure (in minutes)           int64
BMI                                                 float64
Diabetes Mellitus                                  category
Tobacco/Nicotine Use                               category
Functional Heath Status                            category
Hypertension requiring medication                  category
Serum Sodium                                          Int64
BUN                                                 float64
Serum Creatinine                                    float64
Albumin                                             float64
Total Bilirubin                         

,Age at Time of Surgery,Sex at Birth,Race/Ethnicity,CPT Code,Primary Payor Status,ASA Classification,Duration of Surgical Procedure (in minutes),BMI,Diabetes Mellitus,Tobacco/Nicotine Use,Functional Heath Status,Hypertension requiring medication,Serum Sodium,BUN,Serum Creatinine,Albumin,Total Bilirubin,AST/SGOT,Alkaline Phosphatase,WBC,Hematocrit,Platelet Count,INR,PTT,# of Postop Organ/Space SSI PATOS,Surgical Length of Stay,Hospital Discharge Destination,Postoperative ICD10 Code,Height (cm),Weight (kg),Postop Sepsis Occurrence,Postop SSI Occurrence,Readmission likely related to Primary Procedure,INPWT
0,78.71,Female,NaN,48150,NaN,ASA IV - Severe systemic disease threat to life,404,27.99,Non-insulin,No,Independent,Yes,145,19.0,1.57,3.4,0.3,18,92,7.5,24.7,316,1.11,26.4,0,15,Home,C24.1,160.02,71.66,No,No,No,No
1,60.20,Female,NaN,48150,NaN,ASA III - Severe systemic disease,948,30.17,No,No,Independent,Yes,144,15.0,0.81,3.5,2.6,31,198,8.8,32.1,244,1.22,32.1,0,9,Home,C25.3,167.64,84.80,No,No,No,No
2,56.56,Male,NaN,48150,NaN,ASA III - Severe systemic disease,29,34.08,No,Yes,Independent,No,146,9.0,0.82,1.5,8.2,241,992,13.0,24.6,425,1.08,31.8,0,13,Home,C25.3,165.10,92.90,No,No,No,No


## Variable Selection

Candidate predictor variables were screened using two complementary variable selection approaches to identify covariates associated with each outcome. The treatment variable (incisional negative pressure wound therapy) was forced into all models regardless of selection results, as it represented the primary exposure of interest. All other variables underwent selection as potential confounders or effect modifiers.

### Univariate Selection

Each candidate covariate was assessed using univariate statistical tests. Predictors were classified as continuous or categorical based on data type. Continuous predictors were analyzed as numeric values, while categorical predictors included both predefined categories and string-type variables converted to categorical form.

The choice of statistical test was determined by the type of predictor and outcome:

- Continuous outcome, continuous predictor: Pearson correlation.
- Continuous outcome, categorical predictor: Welch's t-test for binary predictors; one-way ANOVA for predictors with more than two categories.
- Categorical outcome, continuous predictor: Welch's t-test for binary outcomes; one-way ANOVA for outcomes with more than two categories.
- Categorical outcome, categorical predictor: Fisher's exact test for 2×2 tables with small expected counts; otherwise, Pearson's chi-square test. Categories with fewer than five observations were combined or excluded to ensure stable estimates.

Observations with missing outcome data were excluded. Missing values within predictor groups were handled by pairwise deletion during testing. Variables with fewer than two unique values were omitted. Covariates yielding a p-value less than 0.1 in univariate testing were identified as candidates for multivariable modeling.

### Elastic Net Selection

As a complementary approach, we used elastic net regularization for variable selection. This method handles multicollinearity by shrinking correlated predictors and automatically performs variable selection by setting coefficients to zero. For binary outcomes, we applied logistic regression with elastic net penalty (L1 ratio = 0.5, balancing Lasso and Ridge penalties). For count outcomes, we applied elastic net regression with the same L1 ratio. Regularization parameters were chosen via 5-fold cross-validation to optimize prediction. Covariates with non-zero coefficients after regularization were identified as candidates for modeling.

#### Repeated k-Fold Stability Assessment

To further assess the robustness of elastic net selection, we implemented a repeated k-fold procedure. For each outcome, the dataset was split into k folds multiple times, elastic net was applied in each fold, and the proportion of folds in which each variable was selected was recorded. Variables selected in more than 50% of folds were considered relatively stable predictors. Additionally, the distribution of their coefficients across folds was summarized to characterize variability and reliability. This approach allows identification of covariates that consistently contribute to prediction, reducing reliance on a single cross-validation split.

### Consensus Selection

To identify the most robust set of covariates, we also planned to fit models using only variables selected by both univariate testing and elastic net (consensus approach). This conservative strategy retained only predictors showing evidence of association through two different methods. When no variables were selected by both methods, the consensus model included only the treatment variable.

The type of test performed and associated p-value or regularization coefficient were recorded for transparency. This dual approach ensured that predictor selection was not dependent on a single statistical method, allowing assessment of whether treatment effects remained consistent across different covariate adjustment strategies.

## Statistical Modeling

### Sample Size and Model Specification

We determined the maximum number of predictors that could be reliably estimated given our sample size, following established guidelines for regression modeling. For binary outcomes, we applied the criterion of 10 events in the minority class per variable (Peduzzi et al., 1996). For count outcomes, we used a more conservative threshold of 15 observations per variable to account for the additional variance and reduced information content per observation in overdispersed count data. The treatment variable counted toward this limit.

When variable selection methods identified more candidate predictors than the calculated maximum, we retained only the top N variables to remain within sample size guidelines. For univariate selection, we retained variables with the smallest p-values. For elastic net selection, we retained variables with the largest absolute coefficients. This approach balanced the information provided by variable selection methods with the statistical requirement for adequate sample size relative to model complexity.

### Separation and Convergence

Prior to model fitting, we assessed all selected variables for quasi-perfect separation—patterns where certain predictor levels were associated with near-complete homogeneity in the outcome. Separation was defined as 95% or more observations in a predictor level falling into a single outcome category. While separation is particularly problematic for logistic regression, where it can cause coefficient estimates to diverge toward infinity, it is less catastrophic for count models. We proceeded with model fitting using standard maximum likelihood estimation regardless of separation patterns. When models failed to converge despite appropriate sample size considerations, we concluded the data were insufficient to support that particular covariate structure and report this limitation explicitly rather than employing alternative estimation methods.

### Model Fitting Strategy

For each outcome, we attempted to fit four models to comprehensively assess treatment effects and potential confounding:

1. **Unadjusted model**: Treatment variable only, using all available observations with complete outcome data
2. **Univariate-adjusted model**: Treatment plus covariates selected by univariate testing
3. **Elastic net-adjusted model**: Treatment plus covariates selected by elastic net regularization, with repeated k-fold stability assessment to identify robust predictors
4. **Consensus-adjusted model**: Treatment plus covariates selected by both methods

This strategy allowed comparison of unadjusted and adjusted estimates to evaluate confounding. When adjusted models failed to converge due to separation or other numerical issues, unadjusted estimates provided important context about marginal treatment associations, with appropriate caveats about potential unmeasured or unmeasurable confounding.


### Binary Outcomes

Postoperative sepsis, surgical site infection, and readmission related to the primary procedure were treated as binary outcomes. Prior to modeling, continuous predictors were standardized to facilitate comparison of effect sizes and improve convergence. Categorical variables were one-hot encoded with the first category dropped as reference, and rare categories (fewer than 5 observations) were excluded to avoid sparse cells. Observations with missing data in either the outcome or selected predictors were removed via listwise deletion.

### Count Outcomes

Surgical length of stay was measured in days and treated as a count outcome. Observed variance substantially exceeded the mean, indicating overdispersion; therefore, negative binomial regression was used rather than Poisson regression. Continuous predictors were not standardized for count models, as negative binomial regression coefficients have natural interpretability on the log scale. Categorical encoding and missing data handling followed the same protocol as binary outcomes.

### Model Reporting

Convergence was assessed for all fitted models. Treatment effects and covariate associations were reported as odds ratios for binary outcomes and incidence rate ratios for length of stay, with corresponding 95% confidence intervals and p-values. By fitting both unadjusted and adjusted models with different selection strategies, we evaluated whether the estimated treatment effect remained consistent across reasonable modeling choices and assessed the degree of confounding by measured covariates. This approach provided evidence for the robustness of findings to covariate adjustment decisions when models converged successfully, and transparently documented limitations when sample size or data structure precluded stable parameter estimation. When adjusted models failed to converge but unadjusted models succeeded, we reported unadjusted estimates with explicit acknowledgment of their limitations regarding causal inference.

In [6]:
from pandas.api.types import CategoricalDtype, is_object_dtype, is_numeric_dtype

response_variables = [col for col in df.columns if metadata[col] == "response"]
print(f"({len(response_variables)}) response variables: {response_variables}")

explanatory_variables = [col for col in df.columns if col not in response_variables]
print(f"({len(explanatory_variables)}) explanatory variables: {explanatory_variables}", end="\n\n")

for response_variable in response_variables:
    dtype = df[response_variable].dtype
    if isinstance(dtype, CategoricalDtype):
        print(df[response_variable].value_counts(dropna=True), end="\n\n")
    elif is_numeric_dtype(df[response_variable]):
        display(df['Surgical Length of Stay'].describe())
    else:
        raise ValueError(f"Column '{response_variable}' must be numeric or categorical, got {dtype}")
    

(4) response variables: ['Surgical Length of Stay', 'Postop Sepsis Occurrence', 'Postop SSI Occurrence', 'Readmission likely related to Primary Procedure']
(30) explanatory variables: ['Age at Time of Surgery', 'Sex at Birth', 'Race/Ethnicity', 'CPT Code', 'Primary Payor Status', 'ASA Classification', 'Duration of Surgical Procedure (in minutes)', 'BMI', 'Diabetes Mellitus', 'Tobacco/Nicotine Use', 'Functional Heath Status', 'Hypertension requiring medication', 'Serum Sodium', 'BUN', 'Serum Creatinine', 'Albumin', 'Total Bilirubin', 'AST/SGOT', 'Alkaline Phosphatase', 'WBC', 'Hematocrit', 'Platelet Count', 'INR', 'PTT', '# of Postop Organ/Space SSI PATOS', 'Hospital Discharge Destination', 'Postoperative ICD10 Code', 'Height (cm)', 'Weight (kg)', 'INPWT']



count        152.0
mean     11.407895
std       5.121511
min            5.0
25%            8.0
50%           10.0
75%           13.0
max           30.0
Name: Surgical Length of Stay, dtype: Float64

Postop Sepsis Occurrence
No     138
Yes     27
Name: count, dtype: int64

Postop SSI Occurrence
No     139
Yes     26
Name: count, dtype: int64

Readmission likely related to Primary Procedure
No     142
Yes     23
Name: count, dtype: int64



In [7]:
def run_analysis(
    df: pd.DataFrame,
    response_variables: list,
    explanatory_variables: list,
    treatment_variable: str = 'INPWT',
    univariate_alpha: float = 0.1,
    epv_binary: int = 10,
    epv_count: int = 15,
    elastic_net_alpha_ratio: float = 0.5,
    elastic_net_inner_cv: int = 5,
    elastic_net_outer_cv: int = 5,
    elastic_net_n_repeats: int = 10,
    elastic_net_selection_threshold: float = 0.5,
):
    """
    Driver function for variable selection + model fitting with EPV-based covariate limiting.
    
    Performs two variable selection methods on covariates:
    1. Univariate selection (p < alpha)
    2. Nested CV Elastic Net selection (consensus variables from repeated CV)
    
    When selection yields more variables than EPV guidelines allow, takes top N by:
    - Univariate: smallest p-values
    - Elastic Net: highest selection frequencies
    
    The treatment variable is ALWAYS included in all models and does NOT count toward EPV limit.
    
    Parameters
    ----------
    df : pd.DataFrame
        Cleaned dataset
    response_variables : list
        List of response variable names
    explanatory_variables : list
        List of candidate predictor names (including treatment)
    treatment_variable : str
        Name of treatment variable to force into all models (default 'INPWT')
    univariate_alpha : float
        Significance level for univariate selection (default 0.1)
    epv_binary : int
        Events per variable threshold for binary outcomes (default 10)
    epv_count : int
        Observations per variable threshold for count outcomes (default 15)
    elastic_net_alpha_ratio : float
        L1 ratio for elastic net (default 0.5)
    elastic_net_inner_cv : int
        Inner CV folds for hyperparameter tuning (default 5)
    elastic_net_outer_cv : int
        Outer CV folds for performance estimation (default 5)
    elastic_net_n_repeats : int
        Number of times to repeat outer CV (default 10)
        With outer_cv=5 and n_repeats=10, yields 50 total iterations
    elastic_net_selection_threshold : float
        Minimum selection frequency for consensus variables (default 0.5)
        Variables selected in ≥50% of CV folds are included
    
    Returns
    -------
    dict
        Model results keyed by response variable and selection method
        Each result includes 'selected_variables' and fitted model
    
    Notes
    -----
    The nested CV elastic net provides:
    - Honest performance estimates on held-out data
    - Variable selection stability across resamples
    - Consensus variables selected in ≥threshold of CV iterations
    
    Recommended settings:
    - Small samples (n<200): outer_cv=5, n_repeats=10 (50 iterations)
    - Large samples (n≥200): outer_cv=10, n_repeats=5 (50 iterations)
    """
    
    # Separate treatment from candidate covariates
    if treatment_variable not in explanatory_variables:
        raise ValueError(f"Treatment variable '{treatment_variable}' not found in explanatory_variables")
    
    candidate_covariates = [v for v in explanatory_variables if v != treatment_variable]
    print(f"Treatment variable: {treatment_variable}")
    print(f"Candidate covariates for selection: {len(candidate_covariates)} variables\n")
    
    model_results = {}
    
    for response_variable in response_variables:
        print(f"\n{'='*80}")
        print(f"Processing response variable: {response_variable}")
        print(f"{'='*80}")
        
        # Determine outcome type and calculate EPV limit
        y_complete = df[response_variable].dropna()
        y_counts = y_complete.value_counts()
        is_binary = len(y_counts) == 2
        outcome_type = 'binary' if is_binary else 'count'
        
        if is_binary:
            n_events = y_counts.min()
            max_variables_total = n_events // epv_binary
            max_covariates = max_variables_total - 1  # Reserve 1 for treatment
            epv_desc = f"{n_events} events / {epv_binary}"
            print(f"\nOutcome type: binary")
            print(f"Event counts: {dict(y_counts)}")
        else:
            n_obs = len(y_complete)
            max_variables_total = n_obs // epv_count
            max_covariates = max_variables_total - 1  # Reserve 1 for treatment
            epv_desc = f"{n_obs} observations / {epv_count}"
            print(f"\nOutcome type: count")
            print(f"Response range: {y_complete.min():.0f} - {y_complete.max():.0f} (mean={y_complete.mean():.1f})")
        
        print(f"EPV calculation: {epv_desc} = {max_variables_total} max variables total")
        print(f"Maximum covariates (excluding treatment): {max_covariates}")
        
        model_results[response_variable] = {}
        
        # =====================================================================
        # METHOD 0: UNADJUSTED (TREATMENT ONLY)
        # =====================================================================
        print(f"\n{'-'*80}")
        print(f"Method 0: Unadjusted (Treatment Only)")
        print(f"{'-'*80}")
        
        print(f"Model variables: [{treatment_variable}]")
        
        # Fit unadjusted model
        result = _fit_model_with_checks(
            df, response_variable, [treatment_variable], outcome_type, is_binary
        )
        
        if result is not None:
            model_results[response_variable]['unadjusted'] = result
        else:
            print(f"Unadjusted model could not be fit for {response_variable}")
        
        # =====================================================================
        # METHOD 1: UNIVARIATE VARIABLE SELECTION
        # =====================================================================
        print(f"\n{'-'*80}")
        print(f"Method 1: Univariate Variable Selection (alpha={univariate_alpha})")
        print(f"{'-'*80}")
        
        selector_univariate = UnivariateVariableSelection(
            df, response_variable, candidate_covariates, alpha=univariate_alpha
        )
        selected_covariates = selector_univariate.selected_explanatory_variables
        
        print(f"Univariate selection identified {len(selected_covariates)} covariates")
        
        # Apply EPV limit
        if len(selected_covariates) > max_covariates:
            print(f"Exceeds EPV limit of {max_covariates}. Selecting top {max_covariates} by p-value...")
            # Get top N by smallest p-value
            top_covariates = (
                selector_univariate.results_df
                .nsmallest(max_covariates, 'p_value')['variable']
                .tolist()
            )
            selected_covariates = top_covariates
        
        print(f"Final covariates: {selected_covariates}")
        
        # Add treatment variable
        univariate_vars = [treatment_variable] + selected_covariates
        print(f"Model variables (treatment + {len(selected_covariates)} covariates): {univariate_vars}")
        
        # Fit model
        result = _fit_model_with_checks(
            df, response_variable, univariate_vars, outcome_type, is_binary
        )
        
        if result is not None:
            model_results[response_variable]['univariate'] = result
        else:
            print(f"Univariate model could not be fit for {response_variable}")
        
        # =====================================================================
        # METHOD 2: NESTED CV ELASTIC NET VARIABLE SELECTION
        # =====================================================================
        print(f"\n{'-'*80}")
        print(f"Method 2: Nested CV Elastic Net Variable Selection")
        print(f"  alpha_ratio={elastic_net_alpha_ratio}, outer_cv={elastic_net_outer_cv}x{elastic_net_n_repeats} repeats")
        print(f"  selection_threshold={elastic_net_selection_threshold*100:.0f}%")
        print(f"{'-'*80}")
        
        selector_elastic = NestedCVElasticNetVariableSelection(
            df=df,
            response_variable=response_variable,
            explanatory_variables=candidate_covariates,
            outcome_type=outcome_type,
            alpha_ratio=elastic_net_alpha_ratio,
            inner_cv=elastic_net_inner_cv,
            outer_cv=elastic_net_outer_cv,
            n_repeats=elastic_net_n_repeats,
            selection_threshold=elastic_net_selection_threshold
        )
        
        # Get consensus variables (selected in ≥threshold of folds)
        selected_elastic_covariates = selector_elastic.selected_explanatory_variables
        
        print(f"\nNested CV identified {len(selected_elastic_covariates)} consensus covariates")
        
        # Print stability report
        selector_elastic.print_stability_report()
        
        # Store selection results for later export
        model_results[response_variable]['elastic_net_stability'] = {
            'selector': selector_elastic,
            'selection_frequencies': selector_elastic.selection_frequencies,
            'coefficient_distributions': selector_elastic.coefficient_distributions,
            'performance_metrics': selector_elastic.performance_metrics,
            'results_df': selector_elastic.get_results_dataframe()
        }
        
        # Apply EPV limit
        if len(selected_elastic_covariates) > max_covariates:
            print(f"\nExceeds EPV limit of {max_covariates}. Selecting top {max_covariates} by selection frequency...")
            # Get top N by highest selection frequency
            results_df = selector_elastic.get_results_dataframe()
            top_covariates = (
                results_df[results_df['consensus_selected']]
                .nlargest(max_covariates, 'selection_frequency')['variable']
                .tolist()
            )
            selected_elastic_covariates = top_covariates
        
        print(f"\nFinal covariates: {selected_elastic_covariates}")
        
        # Add treatment variable
        elastic_vars = [treatment_variable] + selected_elastic_covariates
        print(f"Model variables (treatment + {len(selected_elastic_covariates)} covariates): {elastic_vars}")
        
        # Fit model on full dataset with selected variables
        result = _fit_model_with_checks(
            df, response_variable, elastic_vars, outcome_type, is_binary
        )
        
        if result is not None:
            model_results[response_variable]['elastic_net'] = result
        else:
            print(f"Elastic net model could not be fit for {response_variable}")
        
        # =====================================================================
        # METHOD 3: CONSENSUS (INTERSECTION)
        # =====================================================================
        univariate_fit = 'univariate' in model_results[response_variable]
        elastic_fit = 'elastic_net' in model_results[response_variable]
        
        if univariate_fit and elastic_fit:
            print(f"\n{'-'*80}")
            print(f"Method 3: Consensus Variables (Intersection)")
            print(f"{'-'*80}")
            
            # Get the actual covariates used in each model (excluding treatment)
            univariate_covs = set(selected_covariates)
            elastic_covs = set(selected_elastic_covariates)
            overlap_covariates = univariate_covs & elastic_covs
            
            if len(overlap_covariates) > 0:
                print(f"Covariates selected by BOTH methods: {sorted(overlap_covariates)}")
                consensus_vars = [treatment_variable] + sorted(overlap_covariates)
            else:
                print(f"No covariates selected by both methods.")
                print(f"Fitting treatment-only model.")
                consensus_vars = [treatment_variable]
            
            print(f"Model variables: {consensus_vars}")
            
            # Fit model
            result = _fit_model_with_checks(
                df, response_variable, consensus_vars, outcome_type, is_binary
            )
            
            if result is not None:
                model_results[response_variable]['consensus'] = result
            else:
                print(f"Consensus model could not be fit for {response_variable}")
        
        # =====================================================================
        # COMPARISON SUMMARY
        # =====================================================================
        if univariate_fit and elastic_fit:
            print(f"\n{'-'*80}")
            print(f"Variable Selection Comparison")
            print(f"{'-'*80}")
            
            overlap = univariate_covs & elastic_covs
            univariate_only = univariate_covs - elastic_covs
            elastic_only = elastic_covs - univariate_covs
            
            print(f"Treatment (forced in all models): {treatment_variable}")
            print(f"Overlap ({len(overlap)}): {sorted(overlap) if overlap else 'none'}")
            print(f"Univariate only ({len(univariate_only)}): {sorted(univariate_only) if univariate_only else 'none'}")
            print(f"Elastic net only ({len(elastic_only)}): {sorted(elastic_only) if elastic_only else 'none'}")
            
            # Print selection frequencies for overlapping variables
            if overlap:
                print(f"\nSelection Stability for Overlapping Variables:")
                for var in sorted(overlap):
                    freq = selector_elastic.selection_frequencies.get(var, 0)
                    print(f"  {var}: {freq*100:.0f}% of elastic net iterations")
    
    return model_results


def _fit_model_with_checks(
    df: pd.DataFrame,
    response_variable: str,
    selected_vars: list,
    outcome_type: str,
    is_binary: bool
) -> dict | None:
    """
    Helper function to fit appropriate model with separation checking and convergence handling.
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataset containing response and predictor variables.
    response_variable : str
        Name of response variable.
    selected_vars : list of str
        List of predictor variables to include in model.
    outcome_type : str
        Type of outcome: 'binary' or 'count'.
    is_binary : bool
        Whether outcome is binary.
    
    Returns
    -------
    dict or None
        Dictionary with 'selected_variables', fitted model, and 'converged' flag.
        Returns None if model fitting or convergence failed.
    """
    from statwise.preparation import DataPreparer
    from statwise.modeling import LogisticRegressionModel, NegativeBinomialRegressionModel
    
    print(f"\nPreparing data...")
    
    # Prepare data with separation checking
    preparer = DataPreparer(df, response_variable, selected_vars)
    
    X, y = preparer.preprocess(
        drop_first_category=True,
        scale_numeric=True if is_binary else False,
        min_category_size=5,
        check_separation=True,
        separation_threshold=0.95
    )
    
    print(f"Final preprocessed data: X.shape={X.shape}, y.shape={y.shape}")
    
    # Fit appropriate model
    try:
        if is_binary:
            print("\nFitting logistic regression...")
            model = LogisticRegressionModel()
            model.fit(X, y)
            model_key = 'logistic_regression'
        
        else:  # count outcome
            print("\nFitting negative binomial regression...")
            model = NegativeBinomialRegressionModel()
            model.fit(X, y)
            model_key = 'negative_binomial'
        
        # Check convergence
        if not model.converged:
            print("\n❌ MODEL FAILED TO CONVERGE")
            print("Insufficient data to reliably estimate parameters.")
            return None
        
        print(f"\n✅ Model converged successfully")
        print(model.model.summary())
        
        return {
            'selected_variables': selected_vars,
            model_key: model,
            'converged': True
        }
    
    except Exception as e:
        print(f"\n❌ MODEL FITTING FAILED: {str(e)}")
        return None

In [8]:
model_results = run_analysis(
    df,
    response_variables,
    explanatory_variables,
    treatment_variable="INPWT",
    univariate_alpha=0.1,
    epv_binary=10,
    epv_count=15,
    elastic_net_alpha_ratio=0.5,
    elastic_net_inner_cv=5,        # Inner CV for hyperparameter tuning
    elastic_net_outer_cv=5,        # Outer CV: 5 folds
    elastic_net_n_repeats=10,      # Repeat 10 times = 50 total iterations
    elastic_net_selection_threshold=0.5  # Consensus: ≥50% selection frequency
)

Treatment variable: INPWT
Candidate covariates for selection: 29 variables


Processing response variable: Surgical Length of Stay

Outcome type: count
Response range: 5 - 30 (mean=11.4)
EPV calculation: 152 observations / 15 = 10 max variables total
Maximum covariates (excluding treatment): 9

--------------------------------------------------------------------------------
Method 0: Unadjusted (Treatment Only)
--------------------------------------------------------------------------------
Model variables: [INPWT]

Preparing data...
Checking for separation issues...
Final preprocessed data: X.shape=(163, 1), y.shape=(163,)

Fitting negative binomial regression...
    Results may be unreliable. Check coefficients and standard errors.

❌ MODEL FAILED TO CONVERGE
Insufficient data to reliably estimate parameters.
Unadjusted model could not be fit for Surgical Length of Stay

--------------------------------------------------------------------------------
Method 1: Univariate Variable Sel

## Results

### Overview of Variable Selection and Modeling

We employed a systematic two-stage approach to variable selection and model fitting. First, we applied two independent variable selection methods—univariate testing (p < 0.1) and elastic net regularization (L1 ratio = 0.5, 5-fold cross-validation)—to identify candidate predictors for each outcome. For elastic net, we additionally implemented a repeated k-fold procedure to assess selection stability, recording the proportion of folds in which each variable was selected and the distribution of its coefficients. Variables selected consistently across folds (>50%) were considered more robust predictors.

Second, we determined the maximum number of covariates that could be reliably estimated given our sample size, using established guidelines: 10 events per variable for binary outcomes (Peduzzi et al., 1996) and 15 observations per variable for count outcomes, the latter being more conservative to account for increased variance in count data.

When either selection method identified more variables than the sample size threshold permitted, we retained only the top N variables ranked by statistical strength (smallest p-values for univariate, largest absolute coefficients for elastic net). The treatment variable (incisional negative pressure wound therapy, INPWT) was included in all models regardless of selection and counted toward the variable limit.

We attempted to fit four models per outcome to assess treatment effects and confounding: (1) unadjusted (treatment only), (2) univariate selection (treatment + univariate covariates), (3) elastic net selection (treatment + elastic net covariates), and (4) consensus (treatment + variables selected by both methods). Comparison of unadjusted and adjusted estimates allowed assessment of confounding. When adjusted models failed to converge despite appropriate sample size considerations, we reported unadjusted estimates with appropriate caveats about unmeasured or unmeasurable confounding.

**Table 1. Variable Selection and Model Convergence by Outcome and Method**

| Outcome | Method | Covariates Selected | Model Converged | Notes |
| --- | --- | --- | --- | --- |
| Surgical Length of Stay | Unadjusted | — | ✗ | Failed to converge |
| Surgical Length of Stay | Univariate | 6 | ✓ | — |
| Surgical Length of Stay | Elastic Net | 1 | ✗ | Failed to converge |
| Postop Sepsis Occurrence | Unadjusted | — | ✓ | Treatment-only |
| Postop Sepsis Occurrence | Univariate | 0 | ✓ | Treatment-only |
| Postop Sepsis Occurrence | Elastic Net | 0 | ✓ | Treatment-only |
| Postop Sepsis Occurrence | Consensus | 0 | ✓ | Treatment-only |
| Postop SSI Occurrence | Unadjusted | — | ✓ | Treatment-only |
| Postop SSI Occurrence | Univariate | 1 | ✗ | Failed to converge |
| Postop SSI Occurrence | Elastic Net | 0 | ✓ | Treatment-only |
| Readmission likely related to Primary Procedure | Unadjusted | — | ✓ | Treatment-only |
| Readmission likely related to Primary Procedure | Univariate | 1 | ✗ | Failed to converge |
| Readmission likely related to Primary Procedure | Elastic Net | 0 | ✓ | Treatment-only |

### Surgical Length of Stay

The unadjusted analysis (treatment only) failed to converge, likely due to extreme overdispersion in the full dataset. The univariate method (p < 0.1) selected 6 covariates, all within the sample size limit of 9 covariates (152 observations / 15 = 10 total variables, minus 1 for treatment). This model included 130 patients after excluding those with missing covariate data and converged successfully. Elastic net selected **1 covariate besides treatment** (Serum Creatinine), but the model failed to converge.

**Variables in Univariate Model**

| Variable | Univariate | Elastic Net | Consensus |
| --- | --- | --- | --- |
| Age at Time of Surgery | ✓ | — | — |
| Diabetes Mellitus | ✓ | — | — |
| Duration of Surgical Procedure (in minutes) | ✓ | — | — |
| Hospital Discharge Destination | ✓ | — | — |
| Hypertension requiring medication | ✓ | — | — |
| Primary Payor Status | ✓ | — | — |

Two variables exhibited perfect separation (Hospital Discharge Destination: Acute Care Hospital; Duration of Surgical Procedure: 29 minutes), yet the univariate model converged successfully. This suggests these patterns, while extreme, did not prevent parameter estimation in this specific covariate configuration.

**Repeated k-Fold Elastic Net Stability (Surgical Length of Stay)**

| Variable | Selection Frequency | N-folds Selected | Coef. Mean | Coef. SD | Consensus Selected |
| --- | --- | --- | --- | --- | --- |
| Serum Creatinine | 1.0 | 9 | 0.0905 | 0.1301 | ✓ |
| Duration of Surgical Procedure (in minutes) | 0.444 | 4 | 0.0007 | 0.0003 | ✗ |
| WBC | 0.444 | 4 | -0.0076 | 0.0036 | ✗ |
| AST/SGOT | 0.444 | 4 | -0.0008 | 0.0002 | ✗ |
| Platelet Count | 0.444 | 4 | -0.0003 | 0.0002 | ✗ |
| Hematocrit | 0.222 | 2 | 0.0100 | 0.0079 | ✗ |
| Race/Ethnicity | 0.222 | 2 | — | — | ✗ |
| Age at Time of Surgery | 0.111 | 1 | 0.0022 | 0.0 | ✗ |
| Albumin | 0.111 | 1 | 0.0076 | 0.0 | ✗ |
| Primary Payor Status | 0.111 | 1 | — | — | ✗ |
| Hospital Discharge Destination | 0.111 | 1 | — | — | ✗ |
| INR | 0.111 | 1 | 0.1371 | 0.0 | ✗ |
| PTT | 0.111 | 1 | 0.0024 | 0.0 | ✗ |

Only Serum Creatinine was selected greater than 50% of folds, suggesting it is a relatively stable predictor. Other variables were selected inconsistently, and the failure of the elastic net model to converge indicates that even stable predictors could not overcome separation issues in combination with treatment.

**Treatment Effect on Surgical Length of Stay (Univariate Model)**

| Model | n | Coefficient | IRR | 95% CI | p-value |
| --- | --- | --- | --- | --- | --- |
| Univariate | 130 | -0.098 | 0.91 | 0.78 – 1.05 | 0.190 |

In the adjusted univariate model, INPWT was associated with a 9% reduction in expected length of stay (IRR 0.91, 95% CI 0.78 – 1.05), though this did not reach statistical significance (p = 0.190). The failure of the unadjusted model to converge prevented direct assessment of confounding, though the adjusted estimate suggests a modest protective effect after accounting for discharge destination and diabetes status.

**Key Covariate Effects (Univariate Model)**

| Predictor | Coefficient | IRR | 95% CI | p-value |
| --- | --- | --- | --- | --- |
| Diabetes Mellitus_No | -0.318 | 0.73 | 0.61 – 0.86 | 0.000 |
| Hospital Discharge Destination_Other | 0.349 | 1.42 | 1.14 – 1.75 | 0.001 |
| Diabetes Mellitus_Non-insulin | -0.318 | 0.73 | 0.59 – 0.90 | 0.004 |
| Hospital Discharge Destination_Skilled Care | 0.284 | 1.33 | 1.06 – 1.67 | 0.015 |
| Primary Payor Status_Other | -0.294 | 0.75 | 0.52 – 1.07 | 0.112 |
| Duration of Surgical Procedure (in minutes) | 0.0 | 1.0 | 1.00 – 1.00 | 0.261 |
| Primary Payor Status_Private Insurance | -0.077 | 0.93 | 0.74 – 1.16 | 0.508 |
| Age at Time of Surgery | 0.001 | 1.0 | 0.99 – 1.01 | 0.803 |
| Primary Payor Status_Medicare | 0.03 | 1.03 | 0.81 – 1.31 | 0.808 |
| Hypertension requiring medication_Yes | -0.005 | 0.99 | 0.86 – 1.14 | 0.941 |

Discharge destination was the strongest predictor of length of stay. Patients discharged to other facilities (not home or skilled care) had 42% longer stays (IRR 1.42, 95% CI 1.14-1.75, p = 0.001), and those discharged to skilled care facilities had 33% longer stays (IRR 1.33, 95% CI 1.06-1.67, p = 0.015), compared to discharge home.  

Diabetes status also significantly predicted length of stay. Patients without diabetes had 27% shorter stays compared to insulin-dependent diabetics (IRR 0.73, 95% CI 0.61-0.86, p < 0.000), and those with non-insulin-dependent diabetes had 27% shorter stays (IRR 0.73, 95% CI 0.59-0.90, p = 0.004).


### Postoperative Sepsis

With only 27 sepsis events, the events-per-variable guideline (10 events per variable) permitted a maximum of 2 total variables (treatment + 1 covariate). Neither univariate testing (p < 0.1) nor elastic net regularization identified any covariates. All three models (univariate, elastic net, and consensus) were therefore identical, consisting of the treatment variable only.

**Treatment Effect on Postoperative Sepsis (All Models)**

| Model | n | Events | Coefficient | OR | 95% CI | p-value |
| --- | --- | --- | --- | --- | --- | --- |
| Unadjusted | 163 | 27 | -1.013 | 0.36 | 0.16 – 0.85 | 0.019 |
| Univariate | 163 | 27 | -1.013 | 0.36 | 0.16 – 0.85 | 0.019 |
| Elastic Net | 163 | 27 | -1.013 | 0.36 | 0.16 – 0.85 | 0.019 |
| Consensus | 163 | 27 | -1.013 | 0.36 | 0.16 – 0.85 | 0.019 |

INPWT demonstrated a significant protective association with postoperative sepsis. Patients receiving INPWT had 64% lower odds of developing sepsis (OR 0.36, 95% CI 0.16 – 0.85, p = 0.019). This effect remained robust across all modeling approaches. The absence of selected covariates suggests that, within this dataset and given the statistical power constraints, the treatment effect on sepsis was not confounded by the measured patient or procedural characteristics that were available for analysis.


### Postoperative Surgical Site Infection

With only 26 SSI events, the events-per-variable guideline permitted a maximum of 2 total variables (treatment + 1 covariate). Both univariate testing and elastic net selection identified the same covariate: number of postoperative organ/space SSIs at time of surgery. However, this variable exhibited perfect separation—all patients with organ/space SSI at surgery also had subsequent SSI. This perfect collinearity prevented convergence for both univariate and elastic net approaches.

**Treatment Effect on SSI (Unadjusted Model Only)**

| Model | n | Events | Coefficient | OR | 95% CI | p-value |
| --- | --- | --- | --- | --- | --- | --- |
| Unadjusted | 163 | 25 | -0.067 | 0.94 | 0.39 – 2.23 | 0.880 |

In unadjusted analysis, INPWT showed no evidence of protective effect against SSI (OR 0.94, 95% CI 0.39-2.23, p = 0.880). Although adjusted models could not be fit due to perfect separation in the selected covariate, the null unadjusted finding suggests adjustment is unlikely to reveal a hidden treatment benefit. The inability to adjust for organ/space SSI at surgery—itself a strong predictor—limits causal interpretation, but the wide confidence interval and null point estimate indicate the data provide little evidence for or against a treatment effect.


### Readmission Related to Primary Procedure

With only 23 readmission events, the sample size permitted a maximum of 2 total variables (treatment + 1 covariate). Univariate testing identified 8 potentially associated covariates but was limited to the top 1 by p-value: number of postoperative organ/space SSIs. This variable exhibited perfect separation, causing the univariate model to fail to converge.

Elastic net selected no covariates, allowing a treatment-only model to be fit successfully.

**Treatment Effect on Readmission**

| Model | n | Events | Coefficient | OR | 95% CI | p-value |
| --- | --- | --- | --- | --- | --- | --- |
| Unadjusted | 163 | 23 | -0.024 | 0.98 | 0.40 – 2.41 | 0.959 |
| Elastic Net | 163 | 23 | -0.024 | 0.98 | 0.40 – 2.41 | 0.959 |

No evidence of a treatment effect on readmission was observed (OR 0.98, 95% CI 0.40 – 2.41, p = 0.959). The pseudo R-squared was near zero (0.00002), indicating the treatment explained essentially none of the variation in readmission risk. This null finding should be interpreted cautiously given the small number of events and lack of covariate adjustment.


### Summary

INPWT showed a significant protective association with postoperative sepsis in all models (unadjusted and adjusted were identical as no covariates were selected), with a 64% reduction in odds (OR 0.36, 95% CI 0.16 – 0.85, p = 0.019). This effect was robust and was not confounded by measured patient characteristics that could be included given sample size constraints.

For surgical length of stay, only the adjusted univariate model converged, showing a 9% reduction in expected stay (IRR 0.91, 95% CI 0.78 – 1.05, p = 0.190) that did not reach statistical significance. The unadjusted model failed to converge, likely due to extreme overdispersion in the complete dataset, preventing direct assessment of confounding. The adjusted model identified discharge destination and diabetes status as the strongest predictors of length of stay. Repeated k-fold elastic net highlighted Serum Creatinine as the most stable predictor, though the model failed to converge.

For SSI and readmission, unadjusted analyses showed null findings (SSI: OR 0.94, 95% CI 0.39 – 2.23, p = 0.880; Readmission: OR 0.98, 95% CI 0.40 – 2.41, p = 0.959). Adjusted models could not be fit due to perfect separation in selected covariates or insufficient events. The null unadjusted estimates with point estimates near 1.0 provide no suggestion that adjustment would reveal substantial treatment benefits.

Discharge disposition and diabetes status were the strongest determinants of length of stay in adjusted analysis. Discharge to non-home settings was associated with 33-42% longer stays, and absence of insulin-dependent diabetes was associated with 27% shorter stays compared to insulin-dependent diabetes.

Sample size limitations precluded definitive adjusted analysis of SSI (26 events) and readmission (23 events) outcomes. The small event counts relative to the number of candidate predictors resulted in models that could not be fit due to separation issues. For surgical length of stay, elastic net selection identified 1 stable variable, but convergence failed. Paradoxically, the treatment-only model also failed to converge, highlighting significant overdispersion challenges in count outcome modeling with this dataset.

These findings underscore the importance of sample size considerations in surgical outcomes research and the value of attempting both unadjusted and adjusted analyses. When unadjusted models converge, they provide important context even when adjustment fails. The SSI and readmission results demonstrate that null unadjusted findings, while limited by potential confounding, can still inform clinical decision-making and future research planning. Future studies of these endpoints may require larger samples, alternative analytical approaches such as penalized regression methods specifically designed to handle separation and overdispersion, propensity score methods to reduce dimensionality, or Bayesian approaches that can incorporate prior information to stabilize estimates with sparse data.
